In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import json
from glob import glob
from pathlib import Path
from tqdm import tqdm

In [3]:
base = "data/done-soon/temp/problem_output/"
all_normal_files = glob("data/done-soon/temp/problem_output/*NORMAL.json")
df = pd.DataFrame()
data = []

num_without_search_time = 0
for i, normal in tqdm(enumerate(all_normal_files)):
    mzn = normal[normal.find("MZN-")+4:normal.find("-DZN")] + ".mzn"
    dzn = normal[normal.find("DZN-")+4:normal.find("-OUTPUT")] + ".dzn"

    stats = Path(f"{normal[:-12]}-STATS.json")
    if stats.exists():
        with open(normal, 'r') as normal_output, open(f"{normal[:-12]}-STATS.json", 'r') as stats_output:
            line = normal_output.readline()
            if line: # don't read json from empty output
                normal_time = json.loads(line).get('time')
                stats_all_lines = [json.loads(line).get('statistics') for line in stats_output.readlines()]
                stats = stats_all_lines[-1]

                if normal_time and stats:
                    if "search_time" not in stats.keys():
                        num_without_search_time += 1
                    else:
                        data.append({
                            'normal_time': normal_time * 0.001,
                            'stat_time': stats['search_time'],
                            'problem': normal,
                            'statistics': stats_all_lines,
                            'mzn': mzn,
                            'dzn': dzn
                        })
                        
df = pd.DataFrame(data)
all_data = df
all_data.shape

16814it [04:31, 62.00it/s] 


(14498, 6)

In [7]:
def cleanup(df):
    del df["decision_level_sat"]
    del df["ewma_decision_level_mip"]
    del df["decision_level_mip"]
#     del df["best_objective"]
#     df["unassnVar"]   = (2**df['vars']) - df['opennodes']
#     df["fracFailUnassn"]     = df['conflicts'] / df['unassnVar']         # num failures/ num open nodes
    df["fracOpenVisit"]  = (df['vars'] - df['opennodes']) / df['opennodes']       # ratio of open nodes to visited nodes (how much of soln space explored)
    df["fracBoolVars"]     = df['boolVars'] / df['vars']                 # num bools / total num of vars
    df["fracPropVars"]     = df['propagations'] / df['vars']        # num propagations/ total num of vars
#     df["frac_unassigned"] = df['unassnVar'] / df['vars']  # current assignments/ total vars
    df["fracLongClauses"] = df['long'] + df['bin'] + df['tern']         # fraction of learnt clauses that have more than 3 literals
    df["freqBackjumps"]  = df['back_jumps']/df['search_time']
    return df


def gradients(df_prev, df_curr):
    keys=['conflicts','ewma_conflicts','decisions','search_iterations','opennodes','ewma_opennodes',
          'vars','back_jumps','ewma_back_jumps','solutions','total_time','search_time','intVars',
          'propagations','sat_propagations','ewma_propagations','propagators','boolVars','learnt',
          'bin','tern','long','peak_depth','decision_level_engine','ewma_decision_level_engine',
          'decision_level_treesize','clause_mem','prop_mem','ewma_best_objective',
          'fracOpenVisit','fracBoolVars','fracPropVars','freqBackjumps', 'best_objective']
    for i in keys:
        df_curr[i+'_gradient']=(df_curr[i]-df_prev[i])/0.05*7200
    return df_curr

In [9]:
all_data_filtered = all_data[all_data['normal_time'] > 10]
all_data_filtered.shape

(1132, 6)

In [15]:
features_at_percent = {}

# find index of statistics array at certain percent of TL
def find_index_at_percent(stats, percent):
    left = 0
    right = len(stats) - 1
    while left < right:
        mid = (left + right) // 2
        
        if stats[mid]['search_time'] == percent:
            # Found the first dictionary with target time
            while mid > 0 and stats[mid-1]['search_time'] == percent:
                mid -= 1
            return mid
        elif stats[mid]['search_time'] < percent:
            left = mid + 1
        else:
            right = mid - 1

    return -1

for i in tqdm(range(1,200)):
    df_i=[]
    for id, problem in all_data_filtered.iterrows():
        time_percent = (i / 2) * 7200
        index = find_index_at_percent(problem.statistics, time_percent)
        p = problem.statistics[index]
        # for index, p in enumerate(problem.statistics):
        #     if index == i:
        new_p = dict(p)
        new_p=cleanup(new_p)
        if i!=1:
            new_p=gradients(df_prev.loc[id], new_p)
        new_p['mzn'] = problem['mzn']
        new_p['dzn'] = problem['dzn']
        new_p['solved_within_time_limit'] = problem['normal_time'] < 7199 * 1000 \
        or np.logical_not(np.isnan(problem['normal_time']))
        df_i.append((id, new_p))

    df_i = pd.DataFrame([a[1] for a in df_i], index=[a[0] for a in df_i])
    df_i=df_i.fillna(value = 0)
    if i!=0:   
        features_at_percent[i]=df_i
    df_prev=df_i



  0%|          | 0/199 [00:00<?, ?it/s]